In [ ]:
# Install dependencies
!pip install transformers accelerate bitsandbytes pandas openpyxl --quiet

# Imports
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
faq_df = pd.read_excel("/content/drive/MyDrive/faqs.xlsx")
faq_df.fillna("", inplace=True)  # Prevent NaN errors
faq_df['question'] = faq_df['question'].str.lower().str.strip()
faq_df['keywords'] = faq_df['keywords'].str.lower().str.strip()

In [ ]:
login(token="")#Enter your token here 

# Prepare model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with 8-bit quantization and enable CPU offload for 32-bit modules
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    # Add this parameter to enable offloading 32-bit modules to CPU if necessary
    llm_int8_enable_fp32_cpu_offload=True
)

In [ ]:
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)

# System Prompt
SYSTEM_PROMPT = (
    "You are SS DigiBot, an assistant for SS DigiMark, a digital marketing agency. "
    "Answer user questions in a professional and helpful tone. "
)

# FAQ lookup with keyword fallback
def lookup_faq(user_input):
    user_input_lower = user_input.lower().strip()

    # 1. Exact match (in question)
    for _, row in faq_df.iterrows():
        if user_input_lower in row['question']:
            return row['answer']

    # 2. Keyword match
    for _, row in faq_df.iterrows():
        keywords = [kw.strip() for kw in row['keywords'].split(',')]
        if any(kw in user_input_lower for kw in keywords):
            return row['answer']

    return None

# Fallback to Mistral
def get_mistral_response(user_input):
    prompt = f"<s>[INST] {SYSTEM_PROMPT} {user_input} [/INST]"
    output = chatbot(prompt)[0]['generated_text']
    return output.split("[/INST]")[-1].strip()

# Main response function
def respond(user_input):
    answer = lookup_faq(user_input)
    if answer:
        return answer
    return get_mistral_response(user_input)

# Interface
print("SS DigiBot: Hello! How can I help you? (Type 'exit' to quit)")
while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ['exit', 'quit', 'done']:
        print("SS DigiBot: Goodbye!")
        break
    print("SS DigiBot:", respond(user_input))